https://blog.csdn.net/fengdu78/article/details/142531250

In [3]:
"""
打印onnx节点信息
"""
import onnx
import onnxruntime as rt
import numpy as np
 
def get_layer_output(model, image):
    inputs = []
    outputs = []
    for i,node in enumerate(model.graph.node):
        if i>=1:
            break
        output = node.output[0]
        input = node.input
        inputs.append(input)
        outputs.append(output)
        model.graph.output.extend([onnx.ValueInfoProto(name=output)])  # 列表包含指定节点的输出变量
    ort_session = rt.InferenceSession(model.SerializeToString(), providers=["CPUExecutionProvider"])
    ort_inputs = {}
    for i, input_ele in enumerate(ort_session.get_inputs()):  # 获取输入节点和对应的输入向量
        ort_inputs[input_ele.name] = image[i]
    outputs = [x.name for x in ort_session.get_outputs()]
    outputs = outputs[1:]
    ort_outs = ort_session.run(outputs, ort_inputs)  # 获取节点对应的输出向量
 
    return inputs,outputs,ort_outs
 
 
# 加载ONNX模型
model_path = 'resize_conv_add.onnx'
onnx_model = onnx.load(model_path)
 
intermediate_layer_names = [onnx_model.graph.node[i].name for i in range(len(onnx_model.graph.node))]
print('node=',onnx_model.graph.node)
for node in onnx_model.graph.node:
    print('node_name=',node)
    print('node_input=',node.input)
    print('node_output=',node.output)
#
input_data = np.random.randn(1,3,256,256).astype(np.float32)
roi_data = np.array([]).astype(np.float32)
scales_data = np.array([1,1,2,2]).astype(np.float32)
 
conv_weight_data = np.random.randn(32,3,3,3).astype(np.float32)
conv_bias_data = np.random.randn(32).astype(np.float32)
add_input_data = np.random.randn(1).astype(np.float32)
 
inputs,outputs,ort_outs = get_layer_output(onnx_model,[input_data,roi_data,scales_data,conv_weight_data,conv_bias_data,add_input_data])
print('output_node_shape=',ort_outs[0].shape)
 
# 结果
output_node_shape=(1,3,512,512)

node= [input: "input"
input: "roi"
input: "scales"
output: "conv_input"
name: "resize"
op_type: "Resize"
, input: "conv_input"
input: "conv_weight"
input: "conv_bias"
output: "conv_output"
name: "conv"
op_type: "Conv"
, input: "conv_output"
input: "add_input"
output: "output"
name: "add"
op_type: "Add"
]
node_name= input: "input"
input: "roi"
input: "scales"
output: "conv_input"
name: "resize"
op_type: "Resize"

node_input= ['input', 'roi', 'scales']
node_output= ['conv_input']
node_name= input: "conv_input"
input: "conv_weight"
input: "conv_bias"
output: "conv_output"
name: "conv"
op_type: "Conv"

node_input= ['conv_input', 'conv_weight', 'conv_bias']
node_output= ['conv_output']
node_name= input: "conv_output"
input: "add_input"
output: "output"
name: "add"
op_type: "Add"

node_input= ['conv_output', 'add_input']
node_output= ['output']
output_node_shape= (1, 3, 512, 512)
